# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [8]:
import pandas as pd
data = pd.read_csv('data.csv')

!pip install pymystem3
from pymystem3 import Mystem
m = Mystem()

from IPython.display import display

display(data.info())
print()
display(data.head())
print()
display(data.duplicated().value_counts())
print()
display(data.describe())

Installing mystem to /Users/paantur/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


False    21471
True        54
dtype: int64

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

- В базе данных 12 столбцов, из них: 2 с типом `float62` (вещественные значения), 5 c типом `int62` (целочисленные значения), 5 с типом `object` (строки). 

- Пропущенные значения имеются в столбцах `days_employed` и `total_income`. Пропуски могли появится как случайно (в результате чьей-то ошибки), так и неслучайно (в случае, если человек ни дня не работал или не имел совсем доход).

- В столбце `days_employed` невеверный тип данных (вещественный вместо целочисленного). Ошибка связана с тем, данные в столбце конвертируются в дни из более мелкой величины (из часов например), и соответственно требуется округлить значения до целого.

- В базе данных 54 дупликата. Дуплакаты могли возникнуть как ошибочно (в результате совмещения двух или более баз данных), так и в случае, если каждый дупликат имеет отношение к разным людям.

- В столбце `children` имеются аномальные значения количества детей у заемщика: "-1" и "20". По-видимому, "-1" - это опечатка от "1" (это необходимо исправить), а "20" - это выбросы, которые решено не рассматривать.

- В столбце `days_employed` имеется очень много аномальных значений: как слишком больших, так и отрицательных. Однако нет необходимости эти значения корректировать, так как этот столбец никак не задействован в проекте.

### Шаг 2. Предобработка данных

### Приведение всех данных с текстом к нижнему регистру

In [9]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

### Вывод

Привели все данные с типом `object` к нижнему регистру.

### Обработка пропусков

In [20]:
data['days_employed'] = data['days_employed'].abs()
total_income_grouped = data.groupby('education').agg({'days_employed':'median'})
display(total_income_grouped)
total_income_grouped = data.groupby('income_type').agg({'days_employed':'median'})
display(total_income_grouped)
data['days_employed'].fillna(0, inplace=True)



total_income_grouped = data.groupby('education').agg({'total_income':'median'})
display(total_income_grouped)
total_income_grouped = data.groupby('income_type').agg({'total_income':'median'})
display(total_income_grouped)

for education in data['education'].unique():
    median = data.loc[(data['education'] == education), 'total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['education'] == education), 'total_income'] = median

,days_employed
education,
высшее,1604.5
начальное,2474.0
неоконченное высшее,1049.5
среднее,1970.0
ученая степень,5660.0


,days_employed
income_type,
безработный,366413.0
в декрете,3296.0
госслужащий,2385.0
компаньон,1316.0
пенсионер,360531.0
предприниматель,260.0
сотрудник,1358.5
студент,578.0


,total_income
education,
высшее,175340.818855
начальное,117137.352825
неоконченное высшее,160115.398644
среднее,136478.643244
ученая степень,157259.898555


,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,148978.800029
компаньон,169493.899228
пенсионер,128353.656947
предприниматель,337251.981901
сотрудник,138070.060144
студент,98201.625314


### Вывод

Очевидно, что даже после приведения всех значений в столбце `days_employed` к модулю (избавились от отрицательных значений), значения все равно нереалестичные. Раз столбец не нужен и значения абсолюnно некорректные, заменили пропуски на ноль для того-чтобы в дальнейшем изменить тип данных в этом столбце.

В столбце `total_income` заполнили пропущенные значения сгруппируя заемщиков по уровню образования и опрелением медианного значения по каждой категории. В связи с большим размахом значений ежемесячного дохода в столбце `total_income` от 20.000 до 2.000.000 целесообразно для заполнения пропусков использовать метод определения медианных значений `median()`. Сгруппировали по образованию, потому что на наш взгляд, образование больше влияет на доход, чем тип занятости.

### Замена типа данных

In [11]:
data['days_employed'] = data['days_employed'].astype('int64')

### Вывод

Заменили тип данных в столбце `days_employed` с вещественного на целочисленное. Применили метод `astype()` как наиболее подходящий для данного случая.

### Обработка дубликатов

In [12]:
find_duplicated = data[data.duplicated(keep = False)].sort_values(by = data.columns.values.tolist())
display(find_duplicated)

display(data.duplicated().value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,0,23,среднее,1,не женат / не замужем,4,f,сотрудник,0,136478.643244,сделка с подержанным автомобилем
19321,0,0,23,среднее,1,не женат / не замужем,4,f,сотрудник,0,136478.643244,сделка с подержанным автомобилем
3452,0,0,29,высшее,0,женат / замужем,0,m,сотрудник,0,175340.818855,покупка жилой недвижимости
18328,0,0,29,высшее,0,женат / замужем,0,m,сотрудник,0,175340.818855,покупка жилой недвижимости
4216,0,0,30,среднее,1,женат / замужем,0,m,сотрудник,0,136478.643244,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,0,34,среднее,1,женат / замужем,0,f,сотрудник,0,136478.643244,покупка жилья для сдачи
9013,2,0,36,высшее,0,женат / замужем,0,f,госслужащий,0,175340.818855,получение образования
14432,2,0,36,высшее,0,женат / замужем,0,f,госслужащий,0,175340.818855,получение образования
11033,2,0,39,среднее,1,гражданский брак,1,f,сотрудник,0,136478.643244,сыграть свадьбу


False    21454
True        71
dtype: int64

### Вывод

После приведения всех данных с типом `object` к общему регистру, в базе данных образовалось 71 полных дубликата, но удалять их нецелесообразно, так как строки могут иметь отношения к разным людям. Наглядно дубликаты проверили методом `duplicated()` и сортировкой `sort_values()`.

### Лемматизация

In [13]:
from collections import Counter
Counter(m.lemmatize(' '.join(data['purpose']))) # получили список с количеством лемм в столбце с целями кредита

Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

### Вывод

Определили, что среди целей кредита наиболее распространнными являются: недвижимость, автомобиль, образование, жилье и свадьба. 

### Категоризация данных

In [14]:
education_grouped = data.groupby(['education_id', 'education', 'education']).agg({'count'})
display(education_grouped)
print()
family_status_grouped = data.pivot_table(index=['family_status_id', 'family_status'])
display(family_status_grouped)

,,,children,days_employed,dob_years,family_status,family_status_id,gender,income_type,debt,total_income,purpose
,,,count,count,count,count,count,count,count,count,count,count
education_id,education,education,,,,,,,,,,
0,высшее,высшее,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260
1,среднее,среднее,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233
2,неоконченное высшее,неоконченное высшее,744,744,744,744,744,744,744,744,744,744
3,начальное,начальное,282,282,282,282,282,282,282,282,282,282
4,ученая степень,ученая степень,6,6,6,6,6,6,6,6,6,6


,,children,days_employed,debt,dob_years,education_id,total_income
family_status_id,family_status,,,,,,
0,женат / замужем,0.638207,56986.170598,0.075202,43.564701,0.805250,166762.753578
1,гражданский брак,0.510175,52216.462533,0.092890,42.130476,0.837683,164627.827779
2,вдовец / вдова,0.223958,185286.957292,0.065625,56.513542,0.936458,143366.844633
3,в разводе,0.457741,62366.156485,0.071130,45.517992,0.797490,167677.345414
4,не женат / не замужем,0.286527,42252.849271,0.097405,38.369357,0.807323,166611.947548


### Вывод

По столбцам с категориями (это education_id и family_status_id) составили словари. Для информативности выполнили категоризацию двуми разными методами: в первом случае group_by() и agg(), а во втором - pivot_table().

Как видно из первой таблицы, наиболее часто за кредитом обращаются люди со средним образованием - в три раза чаще, чем люди с высшим образованием. Но скорее всего это следствие того, что людей со средним образованием в принципе больше.

Из второй таблицы можно сделать вывод, что люди из категории "вдовец / вдова" имеют лучшую статистику по возратам кредитов, тогда как люди из категории "гражданский брак" и "Не женат / не замужем" являются аутсайдерами в этой части. 

### Обработка аномальных значений

In [15]:
data['children'] = data['children'].replace(-1, 1)
data = data[data.children != 20]

В столбце `data['children']` заменили количество детей с "-1" на "1", и удалили строки из базы данных с количеством детей равным "20".

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
children_debt = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count'])
children_debt['return'] = children_debt['sum'] / children_debt['count'] * 100
display(children_debt)

,sum,count,return
,debt,debt,
children,,,
0,1063,14149,7.512898
1,445,4865,9.146968
2,194,2055,9.440389
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


### Вывод

Вывели сводную таблицу, отражающую зависимость между количеством детей и возвратом кредита в срок. 

Исходя из сводной таблицы делаем вывод, что на возврат кредита влияет лишь факт наличия детей, а не их количество.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
family_status_debt = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count'])
family_status_debt['return'] = family_status_debt['sum'] / family_status_debt['count'] * 100
display(family_status_debt.sort_values('return'))

,sum,count,return
,debt,debt,
family_status,,,
вдовец / вдова,63,956,6.589958
в разводе,84,1193,7.041073
женат / замужем,928,12331,7.525748
гражданский брак,385,4165,9.243697
не женат / не замужем,273,2804,9.736091


### Вывод

Вывели сводную таблицу, отражающую зависимость между семейным положением и возвратом кредита в срок.

Исходя из сводной таблицы делаем вывод, что на возврат кредита семейное положение влияет. Вероятность возрата кредита в порядке убывания: "вдовец / вдова", "в разводе", "женат / замужем", "гражданский брак", "не женат / не замужем".

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
def income_group(income):
    if income < 100000:
        return 'less than 100000'
    if 100000 <= income < 150000:
        return 'from 100000 to 150000'
    if 150000 <= income < 200000:
        return 'from 150000 to 200000'
    if income >= 200000:
        return 'more than 200000'
data['income_group'] = data['total_income'].apply(income_group)

total_income_debt = data.pivot_table(index = ['income_group'], values = 'debt', aggfunc = ['sum', 'count'])
total_income_debt['return'] = total_income_debt['sum'] / total_income_debt['count'] * 100

display(total_income_debt)

,sum,count,return
,debt,debt,
income_group,,,
from 100000 to 150000,628,7233,8.682428
from 150000 to 200000,395,4718,8.372192
less than 100000,354,4452,7.951482
more than 200000,356,5046,7.055093


### Вывод

Для определения зависимости между уровнем дохода и возвратом кредита в срок сгруппирали ежемесячный доход по группам и вывели сводную таблицу.

Исходя из сводной таблицы делаем вывод, что на возврат кредита доход влияет. Самыми отвественными заемщиками являются те, чей доход составляет более 200т.р. За ними по статистике следуют заемщики с доходом менее 100т.р. На удивление, заемщики с доходом от 100 до 200 т.р. являются менее благонадежными.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
lemmas_list = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']

def main_lemmas(purpose):
    for lemma in lemmas_list:
        if lemma in m.lemmatize(purpose):
            return lemma
data['purpose_category'] = data['purpose'].apply(main_lemmas)

purpose_debt = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['sum', 'count'])
purpose_debt['return'] = purpose_debt['sum'] / purpose_debt['count'] * 100
display(purpose_debt)

,sum,count,return
,debt,debt,
purpose_category,,,
автомобиль,401,4299,9.327751
жилье,308,4458,6.908928
недвижимость,472,6346,7.437756
образование,369,4007,9.208884
свадьба,183,2339,7.823856


### Вывод

Для определения зависимости между целями кредита и его возвратом в срок сгруппирали основные цели по группам и вывели сводную таблицу. Основные цели сгруппировали с помощью лемматизации.

Исходя из сводной таблицы делаем вывод, что на возврат кредита его цели влияют. Так, наибольший процент возвратов по кредитам на "жилье" и "недвижимость", что в принципе является одним и тем же по смыслу. Далее по статистике, в порядке убывания количества возврата кредита в срок, идут такие цели как "свадьба", "образование" и "автомобиль".

### Шаг 4. Общий вывод

В результате проведенного проекта выяснили какие категории людей являются наиболее надежными заемщиками.

Получены ответы на поставленные перед началом проекта задачи:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
Да, есть зависимость, но влияет только факт наличия/отсутствия детей.
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
Да, есть зависимость. Лучше остальных кредиты возращают вдовы и вдовцы.
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Да, есть зависимость. Самыми отвественными заемщиками являются те, чей доход составляет более 200т.р.
- Как разные цели кредита влияют на его возврат в срок?
Да, есть зависимость. Наибольший процент возвратов по кредитам на "жилье" и "недвижимость".

Таким образом, идеальным и наиболее надежным заемщиком явлется человек: не имеющий детей,  овдовевший или в разводе, с уровнем ежемесячного дохода более 200 т.р., а также целью кредита которого является недвижимость.

В отношении базы данных имеются следующие замечания:
- В столбцах `days_employed` и `total_income` имеются пропущенные значения. Их не должно быть, поэтому в случае отсутствия опыта или дохода, просьба указывать нули.
- В базе данных обнаружили 71 дубликат, но их не удалили, так они могут иметь отношение к разным людям
- В столбце `days_employed` очень много аномальных значений, но их необрабатывали, так как данный столбец не использовался в исследовании
- В столбце  `children` имебтся значения количества детей "-1" и "20". В работе решили правильным заменить "-1" на "1", а строки с "20" удалить. Просьба обратить внимание на эти значения.